In [1]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf  # https://pypi.org/project/yfinance/

In [2]:
tickers = [
    "AIAG.L", # L&G Artificial Intelligence UCITS ETF (AIAG.L)
    "NBTK.DE",  # Invesco NASDAQ Biotech UCITS ETF (NBTK.DE)
    "XMLH.DE", # L&G Healthcare Breakthrough UCITS ETF (XMLH.DE)
    "ROBO", # Robo Global Robotics and Automation Index ETF (ROBO)
    "TIUP.DE", # Lyxor Core US TIPS (DR) UCITS ETF (TIUP.DE)
    "VWRL.AS", # Vanguard FTSE All-World UCITS ETF (VWRL.AS)
    "VFEM.AS", # Vanguard FTSE Emerging Markets UCITS ETF (VFEM.AS)
    "DXSA.DE", # Xtrackers Euro Stoxx Quality Dividend UCITS ETF (DXSA.DE)
    "SXR8.DE", # iShares VII PLC - iShares Core S&P 500 UCITS ETF (SXR8.DE)
    "IPRP.AS", # iShares European Property Yield UCITS ETF (IPRP.AS)
    "2B78.DE", # iShares Healthcare Innovation UCITS ETF (2B78.DE)
]
no_years = 5

In [3]:
dfs = []
dates = set() # Keep track of distinct dates in all DataFrames.
# Create a list of DataFrames
for tick in tickers:
    df = yf.Ticker(tick).history(period=f"{no_years}y")
    df["Change"] = df["Close"].pct_change()*100
    df.drop(index=df.index[0], axis=0, inplace=True)  # no change on 1st day
    df.reset_index(inplace=True)  # make idx a col
    df = df[["Date", "Close", "Change"]]
    df["Tick"] = tick
    df["Date"] = df["Date"].dt.date
    dates = dates | set(df["Date"])
    dfs.append(df)

In [4]:
# Make one DataFrame with all tickers' data so calculations can be vectorized:
df = pd.DataFrame(index=sorted(dates))
for ticker_df in dfs:
    tick = ticker_df["Tick"].iloc[0]
    for col in ("Close", "Change"):
        df[f"{tick}_{col}"] = ticker_df.set_index("Date")[col]
df.tail()

,AIAG.L_Close,AIAG.L_Change,NBTK.DE_Close,NBTK.DE_Change,XMLH.DE_Close,XMLH.DE_Change,ROBO_Close,ROBO_Change,TIUP.DE_Close,TIUP.DE_Change,...,VFEM.AS_Close,VFEM.AS_Change,DXSA.DE_Close,DXSA.DE_Change,SXR8.DE_Close,SXR8.DE_Change,IPRP.AS_Close,IPRP.AS_Change,2B78.DE_Close,2B78.DE_Change
2023-05-18,1142.500000,2.816779,39.605000,0.904457,11.282,1.111307,54.549999,1.394052,101.089996,0.262831,...,50.180000,0.139692,20.555000,0.464327,403.559998,1.788281,23.860001,-0.686780,7.028,0.933507
2023-05-19,1138.400024,-0.358860,39.785000,0.454489,11.268,-0.124091,54.779999,0.421631,100.639999,-0.445145,...,49.889999,-0.577921,20.715000,0.778399,402.820007,-0.183366,23.969999,0.461017,7.056,0.398411
2023-05-22,1160.099976,1.906180,40.150002,0.917435,11.478,1.863685,55.279999,0.912742,100.330002,-0.308026,...,50.360001,0.942075,20.745001,0.144826,404.369995,0.384784,24.004999,0.146015,7.179,1.743193
2023-05-23,1420.709961,22.464442,NaN,NaN,NaN,NaN,54.220001,-1.917506,NaN,NaN,...,53.808102,6.846904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-24,1143.500000,-19.512073,39.834999,-0.784564,11.150,-2.857641,53.560001,-1.217263,101.080002,0.747533,...,49.419998,-8.155098,20.360001,-1.855870,397.929993,-1.592601,23.910000,-0.395748,7.002,-2.465525


In [5]:
# Create column to identify ticker with minimum percent change in each row:
pct_change_cols = [c for c in df.columns if c.endswith("_Change")]
df["min_pct_change"] = df[pct_change_cols].idxmin(axis=1)

# Add the close value for the minimum identified ticker:
df["min_ticker_close_value"] = df.apply(
    lambda row: row[row["min_pct_change"].replace("Change", "Close")], axis=1)

df[["min_pct_change", "min_ticker_close_value"]].tail()

,min_pct_change,min_ticker_close_value
2023-05-18,IPRP.AS_Change,23.860001
2023-05-19,VFEM.AS_Change,49.889999
2023-05-22,TIUP.DE_Change,100.330002
2023-05-23,ROBO_Change,54.220001
2023-05-24,AIAG.L_Change,1143.500000


In [6]:
# Divide min_ticker_close_value by daily invest amount to calculate number of shares bought:
daily_invest = 10000 # 100 euros

df["shares_bought"] = daily_invest / df["min_ticker_close_value"]
df[["min_pct_change", "min_ticker_close_value", "shares_bought"]].tail()

,min_pct_change,min_ticker_close_value,shares_bought
2023-05-18,IPRP.AS_Change,23.860001,419.111473
2023-05-19,VFEM.AS_Change,49.889999,200.440973
2023-05-22,TIUP.DE_Change,100.330002,99.671084
2023-05-23,ROBO_Change,54.220001,184.433784
2023-05-24,AIAG.L_Change,1143.500000,8.745081


In [7]:
# Calculate total shares bought in each ticker:
shares_bought = df.groupby(["min_pct_change"])["shares_bought"].sum()
shares_bought = {k.replace("_Change", ""): v for k, v in shares_bought.items()}
shares_bought

{'2B78.DE': 123962.54535624519,
 'AIAG.L': 1317.869944215018,
 'DXSA.DE': 49031.86703309511,
 'IPRP.AS': 56423.91414001852,
 'NBTK.DE': 29272.3992659141,
 'ROBO': 48278.09672329894,
 'SXR8.DE': 1331.723540851829,
 'TIUP.DE': 21745.722089873623,
 'VFEM.AS': 23856.47213781387,
 'VWRL.AS': 972.1147536644552,
 'XMLH.DE': 63386.1338518698}

In [8]:
# Calculate final close prices of tickers:
final_close_prices = {
    k.replace("_Close", ""): v
    for k, v in df.ffill().iloc[-1].items() 
    if "_Close" in k
}
final_close_prices

{'AIAG.L': 1143.5,
 'NBTK.DE': 39.834999084472656,
 'XMLH.DE': 11.149999618530273,
 'ROBO': 53.560001373291016,
 'TIUP.DE': 101.08000183105469,
 'VWRL.AS': 98.66999816894531,
 'VFEM.AS': 49.41999816894531,
 'DXSA.DE': 20.360000610351562,
 'SXR8.DE': 397.92999267578125,
 'IPRP.AS': 23.90999984741211,
 '2B78.DE': 7.001999855041504}

In [9]:
# Calculate final value of holdings:
final_value = {k: v * final_close_prices[k] for k, v in shares_bought.items()}
final_value

{'2B78.DE': 867985.7246150046,
 'AIAG.L': 1506984.281209873,
 'DXSA.DE': 998288.8427204932,
 'IPRP.AS': 1349095.7784782366,
 'NBTK.DE': 1166065.9979580063,
 'ROBO': 2585774.9267997677,
 'SXR8.DE': 529932.7388573338,
 'TIUP.DE': 2198057.628662032,
 'VFEM.AS': 1178986.8093682565,
 'VWRL.AS': 95918.56096407652,
 'XMLH.DE': 706755.3682684571}

In [10]:
total_final_value = sum(final_value.values()) / 100
total_final_value

131838.4665790154

In [11]:
amount_invested = daily_invest * len(df) / 100
amount_invested

129200.0

In [12]:
pct_increase = (total_final_value - amount_invested) / amount_invested
pct_increase

0.02042156794903557